In [ ]:
# %load ../snippets/basic_settings.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import sys
import plotly.express as px
import yaml
import pyranges as pr

sns.set_context("notebook", font_scale=1.1)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)
plt.rcParams["figure.figsize"] = (16, 12)
plt.rcParams['savefig.dpi'] = 200
plt.rcParams['figure.autolayout'] = False
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['text.usetex'] = False  # True activates latex output in fonts!
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.serif'] = "cm"
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x))
from datetime import date
today = date.today().strftime("%d-%m-%y")

In [ ]:
root = Path("/nfs/nas22/fs2202/biol_micro_bioinf_nccr/hardt/nguyenb/tnseq/scratch/03_23_transcriptomics/")

In [ ]:
count_dir = root/"oligo_metag_bowtie_featurecounts"
sample_data_file = root/'rnaseq_metatdata.csv'
sd = pd.read_csv(sample_data_file)

# Annotations

In [ ]:

genome_dict = {"YL32": ("OligoMM12", "CP015399", ".2"),
               "KB18": ("OligoMM12", "CP015400", ".2"),
               "I48": ("OligoMM12", "CP015401", ".2"),
               "YL27": ("OligoMM12", "CP015402", ".2"),
               "YL45": ("OligoMM12", "CP015403", ".2"),
               "I46": ("OligoMM12", "CP015404", ".2"),
               "YL58": ("OligoMM12", "CP015405", ".2"),
               "YL31": ("OligoMM12", "CP015406", ".2"),
               "YL2": ("OligoMM12", "CP015407", ".2"),
               "I49": ("OligoMM12", "CP015408", ".2"),
               "YL44": ("OligoMM12", "CP015409", ".2"),
               "KB1": ("OligoMM12", "CP015410", ".2"),
               "ASF519": ("LCM", "GCF_000364265", ".2"),
               "SL1344": ("Salmonella", "GCA_000210855", ".2"), 
              }
accession_to_name ={}
for k, v in genome_dict.items():
    accession_to_name[v[1]+v[2]]= k
    
accession_to_name['FQ312003.1'] = 'SL1344'
accession_to_name['FQ312003.1;FQ312003.1'] = 'SL1344'
accession_to_name['HE654725.1'] = 'SL1344'
accession_to_name['HE654726.1'] = 'SL1344'
accession_to_name['HE654724.1'] = 'SL1344'

accession_to_name['NZ_AQFV02000001.1'] = "ASF519"
accession_to_name['NZ_AQFV02000002.1'] = "ASF519"
accession_to_name['NZ_AQFV02000003.1'] = "ASF519"
accession_to_name['NZ_AQFV02000004.1'] = "ASF519"
accession_to_name['NZ_AQFV02000005.1'] = "ASF519"
accession_to_name['NZ_AQFV02000006.1'] = "ASF519"
    


In [ ]:

def get_full_annotation(genome_short, genome_long):
    gff_dir = Path("/nfs/nas22/fs2202/biol_micro_sunagawa/Projects/PAN/REFERENCE_GENOMES_PAN/data/raw")
    kegg_dir = Path("/nfs/shared/_shared/lilith/annotated_genomes_oligomm12_asf519")
    gff_file = gff_dir/f"{genome_short}/{genome_long}/{genome_long}.gff3"
    kegg_file_path = kegg_dir/f"{genome_long}/assembly/{genome_long}/kegg/{genome_long}-annotations.tsv"
    if not gff_file.is_file():
        print(gff_file)
        return
    if not kegg_file_path.is_file():
        print(kegg_file_path)
        return
    gff = pr.read_gff3(gff_file).as_df()
    gff = gff[gff.Feature == 'CDS']
    gff['protein_id'] = gff['protein_id'].fillna(gff['locus_tag'])
    kegg_file = pd.read_table(kegg_file_path)
    kegg_file['protein_id'] = kegg_file.QUERY.str.split("prot_", expand=True)[1]
    kegg_file['protein_id'] = kegg_file['protein_id'].apply(lambda x: "_".join(x.split("_")[:-1]))
    full = gff.merge(kegg_file, on='protein_id', how='outer')
    full.to_csv(root/f"{genome_short}.annotation.csv")
    print(genome_short)
    print(full.head())

In [ ]:
# genomes = []
# for value in genome_dict.values():
#     genomes.append((value[1], "".join([value[1], value[2]])))
# genomes = genomes[:-1]
# for genome in genomes:
#     get_full_annotation(*genome)

## Plan

1. How many reads in each file mapped to each genome
    Total numbers:
    
    - median 71% mapped to CDS
    - median of 6 % remains unmapped
    
    Per genome:
    
    - box plot showing distribution of transcriptomes across samples
    
    
2. Saturation curves for each genome 

Conclusion: which genomes can we analyze?

For each genome that we can analyze:
1. PCA: looking for outliers / clustering based on LPS treatment
2. Sanity check: do we see activation of stress response?
3. Run DESeq
    - Can we overlay results on STRING?
    - Add functional annotation from eggNOG for visualisation

## How many reads in each file map to each genome

- Looking at summary files generated by feature counts
- All oligo samples start with AU

In [ ]:
summary_files = count_dir.rglob("*.txt.summary")
summary_files = [f for f in list(summary_files) if 'AU' in f.stem]

In [ ]:
dfs = []
for f in summary_files:
    df = pd.read_table(f)
    df.columns = ['Status', f.stem.split(".count")[0]]
    df = df.set_index("Status").T
    dfs.append(df)
summary_df = pd.concat(dfs).reset_index().rename({'index':'sample_id'}, axis=1).set_index('sample_id')
summary_df = summary_df.loc[:, summary_df.sum(axis=0) > 0].copy()
summary_df['total'] = summary_df.sum(axis=1)
summary_df['perc_assigned'] = summary_df['Assigned']/summary_df['total']*100
summary_df['perc_unmapped'] = summary_df['Unassigned_Unmapped']/summary_df['total']*100
summary_df['perc_no_features'] = summary_df['Unassigned_NoFeatures']/summary_df['total']*100

In [ ]:
summary_df

## Merging featureCounts files

In [ ]:
count_files = count_dir.rglob("*count.txt")
count_files = [f for f in list(count_files) if 'AU' in f.stem]

In [ ]:
annot_df = pd.read_table(count_files[0], comment='#').iloc[:, 0:6].rename({'Geneid':'gene_id'}, axis=1)
annot_df['genome'] = annot_df.Chr.replace(accession_to_name)

In [ ]:
cdfs = []
for f in count_files:
    df = pd.read_table(f, comment="#").iloc[:, [0,6]]
    df.columns = ['gene_id', f.stem.split(".count")[0]]
    df = df.set_index('gene_id')
    cdfs.append(df.T)
count_df = pd.concat(cdfs).T.reset_index().melt(id_vars=['gene_id'], var_name='sample_id', value_name='num_reads')
count_df = count_df.merge(annot_df, on='gene_id')
total_per_genome = count_df.groupby(['sample_id', 'genome']).num_reads.sum().reset_index()
total_per_genome.columns = ['sample_id', 'genome', 'total_per_genome']
totals = (total_per_genome.groupby('sample_id').total_per_genome.sum()
          .reset_index()
          .rename({'total_per_genome': 'total_per_sample'}, axis=1))
total_per_genome = total_per_genome.merge(totals, on='sample_id')
total_per_genome['perc_genome'] = total_per_genome['total_per_genome']/total_per_genome['total_per_sample']*100
total_per_genome['saturation'] = pd.cut(total_per_genome['total_per_genome'], 
                                        [0, 2e6,3e6,1e8], labels=['low', 'med', 'high'])
count_df = count_df.merge(total_per_genome, on=['sample_id', 'genome'])
sd = total_per_genome.merge(sd, on='sample_id', how='left')

In [ ]:
sd.to_csv(root/'08-06-23_summary.csv')

In [ ]:
total_per_genome

In [ ]:
sample_order = sd[sd.genome == 'I48'].sort_values('perc_genome', ascending=False).sample_id.values

In [ ]:
fig = px.bar(sd, x='sample_id', y='perc_genome', color='genome',
             template='plotly_white',
      category_orders= {'genome': ['I48', 'YL32', 'YL58', 'YL27', 'YL44', 'KB18', 'YL45', 'YL31', 'I46',
                       'KB1', 'I49', 'YL2', 'ASF519', 'SL1344'], 
                       'sample_id': sample_order}, hover_data=['Treatment'])

fig.update_xaxes(linewidth=1, linecolor='black',
                ticks="outside", tickwidth=1, title='')
fig.update_yaxes(linewidth=1, linecolor='black', title='Percent of total transcriptome')

fig

In [ ]:
fig = px.box(sd, x='genome', y = 'perc_genome', color='Treatment', 
       points='all', 
       template= 'plotly_white', 
       hover_data=['sample_id', 'total_per_genome'], 
       category_orders= {'genome': ['I48', 'YL32', 'YL58', 'YL27', 'YL44', 'KB18', 'YL45', 'YL31', 'I46',
                       'KB1', 'I49', 'YL2', 'ASF519', 'SL1344']})
       
fig.update_xaxes(linewidth=1, linecolor='black',
                ticks="outside", tickwidth=1, title='')
fig.update_yaxes(linewidth=1, linecolor='black', title='Percent of total transcriptome')

fig

In [ ]:
fig = px.box(sd, x='genome', y = 'total_per_genome', color='Treatment', 
       points='all', 
       template= 'plotly_white', 
       hover_data=['sample_id', 'total_per_genome'], 
       category_orders= {'genome': ['I48', 'YL32', 'YL58', 'YL27', 'YL44', 'KB18', 'YL45', 'YL31', 'I46',
                       'KB1', 'I49', 'YL2', 'ASF519', 'SL1344']})
       
fig.update_xaxes(linewidth=1, linecolor='black',
                ticks="outside", tickwidth=1, title='')
fig.update_yaxes(linewidth=1, linecolor='black', title='Millions of reads')

fig

## Saturation curves

In [ ]:
from numpy.random import RandomState

def rarefy(x, depth=1000, iterations=1, seed=42):
    res = None
    if iterations > 100000:
        print('Max number of iterations allowed is 100000')
        return None
    if iterations > 1:
        seeds = np.random.choice(100000, size=iterations)
    else:
        seeds = [seed]
    for seed in seeds:
        prng = RandomState(seed)
        noccur = np.sum(x)
        nvar = len(x)
        p = x/noccur
        if depth > noccur:
            return np.array([np.nan]*nvar)
        choice = prng.choice(nvar, size=int(depth), p=p)
        if res is None:
            res = np.bincount(choice, minlength=nvar)[np.newaxis,:]
            
        else:
            res = np.concatenate((res, np.bincount(choice, minlength=nvar)[np.newaxis, :]))
    return np.nanmean(res, axis=0)

def rarefy_dataframe(df, depths, seed=0):
    df_list = []
    df.columns.name = 'sample_id'
    for depth in depths: 
        rare_df = df.apply(rarefy, depth=depth, seed=seed).assign(depth=depth)
        df_list.append(rare_df)
        rdf = pd.concat(df_list).reset_index()
        to_drop = rdf.columns[0]
    return rdf.melt(id_vars=[to_drop, 'depth'], var_name='sample_id', value_name='counts')

def saturationCurves(df, depths, cutoffs, seed):
    rareDf = rarefy_dataframe(df, depths, seed).dropna()
    sat_curve_df = (rareDf.groupby(['sample_id', 'depth'])
                    .agg({'counts': [lambda x, c=c: (x > c).sum() for c in cutoffs]})
                    .reset_index())
    sat_curve_df.columns  = ['sample_id', 'depth'] + [f'>{c} reads' for c in cutoffs]
    return rareDf, sat_curve_df

In [ ]:
def saturation_curve_strain(df, strain):
    strain_df = df[df.genome == strain]
    strain_df = strain_df[['gene_id', 'sample_id', 'num_reads']].pivot(index='gene_id', columns='sample_id')
    strain_df.columns = [f[1] for f in strain_df.columns]
    rare_df, sat_curve_df = saturationCurves(strain_df, [1e5,3e5, 5e5,7e5, 1e6,1.5e6, 2e6,
                                                    3e6, 4e6, 5e6, 6e6, 7e6, 8e6], [5], 9)
    fig = px.line(sat_curve_df, x='depth', y='>5 reads', color='sample_id',
       template='plotly_white', title=strain)
    fig.add_vline(x=3000000, line_width=1, line_dash="dash")
    return fig

In [ ]:
saturation_curve_strain(count_df, 'YL32')

In [ ]:
saturation_curve_strain(count_df, 'I48')

In [ ]:
saturation_curve_strain(count_df, 'YL58')

In [ ]:
saturation_curve_strain(count_df, 'YL27')

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
def find_pcs(df, num_pcs=2, num_genes=500, choose_by='variance'):
    """
    :param numPCs:
    :param numGenes:
    :return:
    """
    if num_genes:
        # calculate var for each, pick numGenes top var across samples -> df
        if choose_by == 'variance':
            genes = df.var(axis=1).sort_values(ascending=False).head(num_genes).index
            fdf = df.loc[genes].T
        else:
            pass
            # todo implement log2fc selection
    else:
        fdf = df.T
    pca = PCA(n_components=num_pcs)
    principal_components = pca.fit_transform(fdf)
    pcs = [f'PC{i}' for i in range(1, num_pcs + 1)]
    pc_df = (pd.DataFrame(data=principal_components, columns=pcs).set_index(fdf.index))
    pc_var = {pcs[i]: round(pca.explained_variance_ratio_[i] * 100, 2) for i in range(0, num_pcs)}
    #pc_df = pc_df.merge(_self.sd, left_index=True, right_index=True)
    return pc_df, pc_var

def get_strain_pca(df, strain, sd, color_by='Treatment', symbol_by='saturation'):
    strain_df = df[df.genome == strain]
    strain_df = strain_df[['gene_id', 'sample_id', 'num_reads']].pivot(index='gene_id', columns='sample_id')
    strain_df.columns = [f[1] for f in strain_df.columns]
    strain_df = np.log2(strain_df/strain_df.sum()*1000000 + 0.5)
    pc_df, pc_var = find_pcs(strain_df, 2, 1000)
    pc_df = pc_df.reset_index().rename({'index':'sample_id'}, axis=1)
    pc_df = pc_df.merge(sd[sd.genome == strain], on='sample_id', how='left')
    fig = px.scatter(pc_df, x='PC1', y='PC2', color=color_by, symbol=symbol_by,
                    template='plotly_white', width=800, height=800, 
                     hover_data=[c for c in pc_df.columns if 'PC' not in c], title=strain)
    fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'), )
    return fig, pc_df

In [ ]:
i48_d = sd[sd.genome == 'I48'].copy()
i48_d['i48_dominant'] = i48_d.perc_genome > 49
sd = sd.merge(i48_d[['sample_id', 'i48_dominant']], on='sample_id')

In [ ]:
sample_info = """
AU647
Oligo
PBS
1
AU648
Oligo
PBS
1
AU649
Oligo
PBS
1
AU650
Oligo
LPS
2
AU651
Oligo
LPS
2
AU652
Oligo
LPS
2
AU653
Oligo
LPS
2
AU654
Oligo
PBS
3
AU655
Oligo
PBS
3
AU656
Oligo
PBS
3
AU657
Oligo
LPS
4
AU658
Oligo
LPS
4
""".strip().split("\n")
sample_info = pd.DataFrame(np.array(sample_info).reshape(12, 4), columns=['sample_id', 'Mouse', 'Treatment', 'Cage'])

In [ ]:
sd = sd.merge(sample_info[['sample_id', "Cage"]], on='sample_id')

In [ ]:
sd

In [ ]:
fig, pc_df = get_strain_pca(count_df, 'YL32', sd)

In [ ]:
fig

In [ ]:
fig, pc_df = get_strain_pca(count_df, 'I48', sd, symbol_by='i48_dominant')
fig

In [ ]:
fig, pc_df = get_strain_pca(count_df, 'YL58', sd)
fig

In [ ]:
fig, pc_df = get_strain_pca(count_df, 'YL27', sd)
fig

In [ ]:
fig, pc_df = get_strain_pca(count_df, 'YL44', sd)
fig

## Writing out datasets

In [ ]:
def get_strain_df(df, strain, saturation='low'):
    strain_df = df[df.genome == strain]
    if saturation not in ['low', 'med', 'high']:
        return None
    if saturation == 'med':
        strain_df = strain_df[strain_df.saturation != 'low']
    elif saturation == 'high':
        strain_df = strain_df[strain_df.saturation == 'high']
    strain_df = strain_df[['gene_id', 'sample_id', 'num_reads']].pivot(index='gene_id', columns='sample_id')
    strain_df.columns = [f[1] for f in strain_df.columns]
    return strain_df

In [ ]:
count_df.sample(10)

In [ ]:
yl32 = get_strain_df(count_df, 'YL32')
yl32.to_csv(count_dir/f"{today}_YL32_all_samples.csv")

i48 = get_strain_df(count_df, 'I48', saturation='high')
i48.to_csv(count_dir/f"{today}_I48_all_samples.csv")

yl58 = get_strain_df(count_df, 'YL58')
yl58.to_csv(count_dir/f"{today}_YL58_all_samples.csv")

yl27 = get_strain_df(count_df, 'YL27')
yl27.to_csv(count_dir/f"{today}_YL27_all_samples.csv")

In [ ]:
yl32 = get_strain_df(count_df, 'YL32', 'high')
yl32.to_csv(count_dir/f"{today}_YL32_high_samples.csv")

i48 = get_strain_df(count_df, 'I48', 'high')
i48.to_csv(count_dir/f"{today}_I48_high_samples.csv")

yl58 = get_strain_df(count_df, 'YL58', 'high')
yl58.to_csv(count_dir/f"{today}_YL58_high_samples.csv")

yl27 = get_strain_df(count_df, 'YL27', 'high')
yl27.to_csv(count_dir/f"{today}_YL27_high_samples.csv")

In [ ]:
yl32 = get_strain_df(count_df, 'YL32', 'med')
yl32.to_csv(count_dir/f"{today}_YL32_med_samples.csv")

i48 = get_strain_df(count_df, 'I48', 'med')
i48.to_csv(count_dir/f"{today}_I48_med_samples.csv")

yl58 = get_strain_df(count_df, 'YL58', 'med')
yl58.to_csv(count_dir/f"{today}_YL58_med_samples.csv")

yl27 = get_strain_df(count_df, 'YL27', 'med')
yl27.to_csv(count_dir/f"{today}_YL27_med_samples.csv")

# First results

In [ ]:
i48_res = pd.read_csv(root/"oligo_metag_bowtie_featurecounts/results/02-06-23-I48-PBS_vs_LPS_l0.5a0.01_results.csv")

In [ ]:
ups = i48_res[(i48_res.log2FoldChange > 1) & (i48_res.padj < 0.01)].gene_id.values

In [ ]:
downs = i48_res[(i48_res.log2FoldChange < -1) & (i48_res.padj < 0.01)].gene_id.values

In [ ]:
ups = [u.split('-')[1] for u in ups]
downs = [d.split('-')[1] for d in downs]

In [ ]:
print("\n".join(ups))

In [ ]:
print("\n".join(downs))